# install cool stuff

In [1]:
!pip install geopandas

# import stuff

In [3]:
import json
from shapely.geometry import Polygon

# Load the points geo json data as a geo dataframe

In [20]:
import plotly.figure_factory as ff
import plotly.express as px
import geopandas

filename = "dk_pop_1k"

points_df = geopandas.read_file(f'../geojson_files/{filename}_points.geojson')

points_df

,data,geometry
0,0.218567,POINT (10.58625 57.75375)
1,0.512380,POINT (10.59458 57.75375)
2,0.716782,POINT (10.60292 57.75375)
3,0.992388,POINT (10.61125 57.75375)
4,0.872486,POINT (10.61958 57.75375)
...,...,...
94672,0.296320,POINT (11.94458 54.56208)
94673,2.932083,POINT (11.95292 54.56208)
94674,1.779469,POINT (11.96125 54.56208)
94675,3.801397,POINT (11.96958 54.56208)


In [6]:
len(points_df.index)

94677

In [7]:
points_df['lon'] = points_df['geometry'].x
points_df['lat'] = points_df['geometry'].y

points_df

,data,geometry,lon,lat
0,0.218567,POINT (10.58625 57.75375),10.586250,57.753750
1,0.512380,POINT (10.59458 57.75375),10.594583,57.753750
2,0.716782,POINT (10.60292 57.75375),10.602917,57.753750
3,0.992388,POINT (10.61125 57.75375),10.611250,57.753750
4,0.872486,POINT (10.61958 57.75375),10.619583,57.753750
...,...,...,...,...
94672,0.296320,POINT (11.94458 54.56208),11.944583,54.562083
94673,2.932083,POINT (11.95292 54.56208),11.952917,54.562083
94674,1.779469,POINT (11.96125 54.56208),11.961250,54.562083
94675,3.801397,POINT (11.96958 54.56208),11.969583,54.562083


In [8]:
records = points_df[["lon", "lat"]].to_records(index=False)
lst = list(records)

#for e in list(map(lambda x: x[1], lst)):
#    print(e)

### The current polygons are squares, so we calculate the distance between points and set that as the height and widht of the polygons

In [12]:
curr = lst[0]

dct = {}

#go through all points and accumulate all unique distances between adjacent points in latitude (y axis but geodata)
# either go through all elemnts or max 5000

for i in range(1, min(len(lst),5000)):
    next = lst[i]
    
    if next[1] != curr[1]: # new line
        diff = curr[1] - next[1]
        if diff not in dct:
            dct[diff] = 1
        else: 
            dct[diff] = dct[diff] + 1
    
    curr = next
    
#create a dictionary of unique height differences

In [13]:
# set height and width of the polygons to be the biggest size difference between all points
max_diff = max(dct, key=dct.get)

max_diff

0.0083333333000013

In [14]:
height = max_diff
width = max_diff

In [15]:
poly_df = geopandas.GeoDataFrame({})
def createPolygon(row):
    # create polygon(square) around points by using width and height
    points = [
        (row.lon - width/2, row.lat - height/2),
        (row.lon + width/2, row.lat - height/2),
        (row.lon + width/2, row.lat + height/2),
        (row.lon - width/2, row.lat + height/2),
        (row.lon - width/2, row.lat - height/2)
    ]
    return Polygon(points)



poly_df["geometry"] = points_df.apply(lambda row: createPolygon(row), axis = 1)
poly_df["population_density"] = points_df["data"]
poly_df["lon"] = points_df["lon"]
poly_df["lat"] = points_df["lat"]
poly_df["id"] = [i for i in range(len(points_df.index))]

poly_df.crs = points_df.crs

poly_df

,geometry,population_density,lon,lat,id
0,"POLYGON ((10.58208 57.74958, 10.59042 57.74958...",0.218567,10.586250,57.753750,0
1,"POLYGON ((10.59042 57.74958, 10.59875 57.74958...",0.512380,10.594583,57.753750,1
2,"POLYGON ((10.59875 57.74958, 10.60708 57.74958...",0.716782,10.602917,57.753750,2
3,"POLYGON ((10.60708 57.74958, 10.61542 57.74958...",0.992388,10.611250,57.753750,3
4,"POLYGON ((10.61542 57.74958, 10.62375 57.74958...",0.872486,10.619583,57.753750,4
...,...,...,...,...,...
94672,"POLYGON ((11.94042 54.55792, 11.94875 54.55792...",0.296320,11.944583,54.562083,94672
94673,"POLYGON ((11.94875 54.55792, 11.95708 54.55792...",2.932083,11.952917,54.562083,94673
94674,"POLYGON ((11.95708 54.55792, 11.96542 54.55792...",1.779469,11.961250,54.562083,94674
94675,"POLYGON ((11.96542 54.55792, 11.97375 54.55792...",3.801397,11.969583,54.562083,94675


In [21]:
poly_df.to_file(f'../geojson_files/{filename}_polygons.geojson', driver="GeoJSON")
#poly_df.to_csv("dk_polygons_1k.csv")